
# Lab 3

Solve efficiently a generic n**(2-1) puzzle (also known as Gem Puzzle, Boss Puzzle, Mystic Square, etc.) using path-search algorithms
- Quality: number of actions in the solution
- Cost: total number of actions evaluated
- Efficiency: Quality vs. Cost

DEADLINE: Sunday, November 24 @23:59 UTC


In [ ]:
import random
import heapq
import time
import tracemalloc

# A* helper functions

In [ ]:


def flatten(board):
    """Convert the 2D board (list of lists) to a 1D list"""
    return [tile for row in board for tile in row]


def unflatten(state, size):
    """Convert the 1D list back to a 2D board (list of lists)"""
    return [state[i:i+size] for i in range(0, len(state), size)]


def count_inversions(state):
    """Count the number of inversions in the flattened state"""
    inversions = 0
    n = len(state)
    # Loop through each pair of tiles to count how many inversions exist
    for i in range(n):
        for j in range(i + 1, n):
            # An inversion occurs when a larger number appears before a smaller number
            if state[i] > state[j] and state[i] != 0 and state[j] != 0:
                inversions += 1
    return inversions


def is_solvable(board, n):
    """Check if the n-puzzle is solvable""" 
    flat_state = flatten(board)  # Flatten the board into a 1D list
    inversions = count_inversions(flat_state)  # Count the number of inversions

    if n % 2 == 1:
        # For odd n, puzzle is solvable if inversions are even
        return inversions % 2 == 0
    else:
        # For even n, solvability depends on the row of the blank (0) tile
        blank_row = flat_state.index(0) // n  # Row index of the blank tile (0-indexed)
        if blank_row % 2 == 0:
            # If blank tile is on an even row (0-indexed), inversions must be odd
            return inversions % 2 != 0
        else:
            # If blank tile is on an odd row (0-indexed), inversions must be even
            return inversions % 2 == 0


def get_goal_state(size):
    """Return the goal state for the given size"""
    # The goal state is a list from 1 to size*size-1 followed by 0 (blank tile)
    return list(range(1, size * size)) + [0]

# Heuristics Used in the Code

The code is designed to operate using three primary heuristics, each serving to estimate the cost of reaching the goal state in a search problem. These heuristics are:

1. **Manhattan Distance**: This measures the total distance each tile is from its goal position, calculated as the sum of horizontal and vertical moves required. It provides a straightforward and efficient way to gauge how "far off" the current state is.

2. **Tiles Out of Place**: This heuristic counts the number of tiles that are not in their correct positions. While simpler than the Manhattan distance, it can give a quick overview of the discrepancy between the current state and the goal.

3. **Linear Conflict**: This heuristic builds on the Manhattan distance by identifying pairs of tiles that are in the same row or column as their goal positions but are incorrectly ordered. Resolving such conflicts typically requires additional moves, making this a useful refinement for more accurate cost estimation.

The A* and IDA* algorithms leverage these heuristics in two distinct ways, depending on whether the focus is on exploration or exploitation:

- Exploration: During the exploratory phase, the algorithms prioritize broader searches through the state space. In this phase, a weighted sum of the Manhattan distance and the Tiles Out of Place heuristic is used. This combination provides a balance between accuracy and computational efficiency, guiding the search towards promising regions without becoming overly specific.

- Exploitation: In contrast, during the exploitation phase, the algorithms concentrate on refining and narrowing the search within the most promising paths. Here, a weighted sum of the Manhattan distance and the Linear Conflict heuristic is employed. This approach leverages the added precision of the Linear Conflict heuristic to resolve specific challenges and ensure the search progresses optimally toward the goal state.

By adjusting the weights of the heuristics used in the A* and IDA* algorithms, the results may vary significantly.

In [ ]:
# Manhattan distance heuristic to estimate the cost from the current state to the goal state
def manhattan_distance(state, goal, size):  
    """Calculate the Manhattan distance of a state."""  
    distance = 0
    for i in range(1, size * size):  # For each number except 0 (the empty tile)
        x1, y1 = divmod(state.index(i), size)  # Current position of the tile (i)
        x2, y2 = divmod(goal.index(i), size)   # Goal position of the tile (i)
        # Add the absolute difference in x and y positions to the total distance
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# Tiles out of place heuristic to estimate the cost from the current state to the goal state
def tiles_out_of_place(state, goal, size):    
    """Calculate the number of tiles out of place in a state."""
    distance = 0
    for i in range(1, size * size):  # For each number except 0 (the empty tile)
        if state.index(i) != goal.index(i):  # If the tile is not in its goal position
            distance += 1
    return distance

# Linear conflict heuristic to estimate the cost from the current state to the goal state
def linear_conflict(state, goal, size, md = 0):    
    """Calculate the linear conflict of a state."""
    def find_linear_conflicts(line, goal_line):
        """Find linear conflicts in a single row or column."""
        conflicts = 0
        for i in range(len(line)):
            if line[i] in goal_line and line[i] != 0:  # Tile must belong to this line
                for j in range(i + 1, len(line)):
                    if line[j] in goal_line and line[j] != 0 and goal_line.index(line[i]) > goal_line.index(line[j]):
                        conflicts += 1
        return conflicts

    if md != 0:
        distance = md
    else:
        distance = manhattan_distance(state, goal, size)  # Start with Manhattan distance

    # Check rows for linear conflicts
    for row in range(size):
        current_row = state[row * size:(row + 1) * size]
        goal_row = goal[row * size:(row + 1) * size]
        distance += 2 * find_linear_conflicts(current_row, goal_row)

    # Check columns for linear conflicts
    for col in range(size):
        current_col = [state[row * size + col] for row in range(size)]
        goal_col = [goal[row * size + col] for row in range(size)]
        distance += 2 * find_linear_conflicts(current_col, goal_col)

    return distance

def exploration(state, goal, size):
    """
    Calculate the cost of a state using the exploration heuristic.
    The exploration heuristic is a weighted sum of Manhattan distance and tiles out of place.
    """
    W_MANHATTAN = 0.2
    W_TILES_OUT_OF_PLACE = 0.8

    cost_manahattan = manhattan_distance(state, goal, size)
    cost_tiles_out_of_place = tiles_out_of_place(state, goal, size)

    cost = W_MANHATTAN * cost_manahattan + W_TILES_OUT_OF_PLACE * cost_tiles_out_of_place

    return cost


def exploitation(state, goal, size):
    """
    Calculate the cost of a state using the exploitation heuristic.
    The exploitation heuristic is a weighted sum of Manhattan distance and linear conflict
    """

    W_MANHATTAN = 1
    W_LINEAR_CONFLICT = 1

    cost_manahattan = manhattan_distance(state, goal, size)
    cost_linear_conflict = linear_conflict(state, goal, size, cost_manahattan)

    cost = W_MANHATTAN * cost_manahattan + W_LINEAR_CONFLICT * cost_linear_conflict

    return cost

### A* Algorithm Overview

The A* algorithm is implemented to solve puzzles by dynamically leveraging both **exploration** and **exploitation** strategies. Its approach varies depending on the puzzle size, ensuring optimal performance while maintaining computational efficiency.

1. **For Small Puzzles (less than 4x4)**:  
   When dealing with puzzles smaller than 4x4, the algorithm relies exclusively on **exploitation**. Given the reduced complexity of such puzzles, this approach focuses on finding the best optimal path without the need for broader exploratory techniques. The computation required is minimal, and the solution is achieved efficiently.

2. **For Larger Puzzles (5x5 and above)**:  
   For puzzles of 5x5 or larger, the algorithm incorporates a combination of **exploration** and **exploitation** using a **simulated annealing approach**. This strategy introduces a temperature parameter to control the balance:
   - **High temperature**: Increases the likelihood of **exploration**, allowing the algorithm to search more broadly and escape local optima.
   - **Low temperature**: Focuses on **exploitation**, directing the search toward the most promising paths for a more refined and targeted solution.

In [ ]:


def heuristic_a(state, goal, size, t):
    """
    Calculate the cost of a state using the exploration-exploitation heuristic.
    The exploration-exploitation heuristic is a random choice based on the temperature between exploration and exploitation.
    """
    # op[0] = exploration, op[1] = exploitation
    r = random.random()
    if t > r:
        op = (1,0)
        cost = exploration(state, goal, size)
    else:
        op = (0,1)
        cost = exploitation(state, goal, size)
    return cost, op



def solve_puzzle_a(initial_state):
    """ Puzzle solver using A* algorithm with exploration-exploitation heuristic """

    # starting value for exploration-exploitation balance
    # if n >= BALANCE_E_E, we will use exploration-exploitation heuristic, else only exploitation is used
    BALANCE_E_E = 5 

    # Start the memory profiler
    tracemalloc.start()

    n = len(initial_state)  # Size of the grid (e.g., 3 for a 3x3 puzzle)
    nn = n * n  # Total number of tiles in the grid

    # Construct the goal state dynamically based on the grid size
    goal_state = get_goal_state(n)

    # If the initial state is already the goal, return an empty path
    if initial_state == goal_state:
        return []

    # Check if the puzzle is solvable; if not, return None
    if is_solvable(initial_state, n) is not True:
        print("Puzzle is unsolvable!")
        return None

    # Flatten the initial board for processing
    initial_state_1d = flatten(initial_state)

    # Priority queue (pq) holds states along with their costs: (cost, state, g, prev_empty, path)
    pq = []
    heapq.heappush(pq, (0, initial_state_1d, 0, -1, []))  # Initial cost is 0, no path yet
    visited = set()  # Set to store visited states

    action_count = 0  # Counter to keep track of the number of actions taken

    # Directions for moving: left, right, up, down
    directions = {(-1): 'left', (1): 'right', (-n): 'up', (n): 'down'}

    # If n >= BALANCE_E_E, setup the parameters for exploration-exploitation heuristic
    if n >= BALANCE_E_E:

        TEMP = 1000 *n
        COOLING_RATE = 0.99
        MIN_T = 0.001
        b = True
            
        if n >= 6: # with bigger n, we need to favour exploration more
            MIN_T = 0.01
        
        print("Initial temperature: " + str(TEMP))
        print("Cooling rate: " + str(COOLING_RATE))
        print("Minimum temperature: " + str(MIN_T))

    # Counters for exploration and exploitation
    exploration_count = 0
    exploitation_count = 0
    
    while pq:
        # Pop the state with the lowest cost from the priority queue
        cost, state, g, prev_empty, path = heapq.heappop(pq)

        action_count += 1  # Increment the action count
        if action_count % 500_000 == 0:
            current, peak = tracemalloc.get_traced_memory()
            print("N: " ,n," Action count: " + str(action_count), " Current memory usage is " + str(current / 10**6) + "MB; Peak was " + str(peak / 10**6) + "MB")

        # If the current state matches the goal state, return the path
        if state == goal_state:
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
            return path, action_count, exploration_count, exploitation_count, current

        visited.add(hash(tuple(state)))  # Mark the state as visited

        empty = state.index(0)  # Find the position of the empty tile (0)

        # Explore the 4 possible moves: left, right, up, down
        moves = [-1, 1, -n, n]  # Offsets for left, right, up, down
        for move in moves:
            new_empty = empty + move  # Calculate the new position of the empty tile

            # Skip invalid moves (out of bounds or invalid positions)
            if new_empty < 0 or new_empty >= nn or (empty % n == 0 and move == -1) or (empty % n == n - 1 and move == 1):
                continue

            # Create a new state by swapping the empty space with the target tile
            new_state = state[:]
            new_state[empty], new_state[new_empty] = new_state[new_empty], new_state[empty]

            # Skip if the new state has been visited
            if hash(tuple(new_state)) in visited:
                continue
            
            if n >= BALANCE_E_E:
                # Cooling down the temperature
                if(TEMP > MIN_T):
                    TEMP *= COOLING_RATE
                elif b:
                    print("T < MIN_T, stop cooling down, action count: " + str(action_count))
                    b = False

                # Calculate the heuristic cost for the new state using exploration-exploitation heuristic
                cost, op = heuristic_a(new_state, goal_state, n, TEMP)
                exploration_count += op[0]
                exploitation_count += op[1]
                h = cost
            
            else:
                # Calculate the heuristic cost for the new state using only exploitation heuristic
                h = exploitation(new_state, goal_state, n)
                exploitation_count += 1

            new_move = directions[move]  # Get the move direction (left, right, up, down)

            # Push the new state into the priority queue with updated cost and path
            heapq.heappush(pq, (g + 1 + h, new_state, g + 1, empty, path + [new_move]))

    return None  # Return None if no solution is found


# IDA* Algorithm Overview

The IDA* (Iterative Deepening A*) algorithm is particularly suited for solving larger puzzles, especially those greater than 6x6, where memory usage is a concern. Unlike A*, which stores all nodes in memory, IDA* minimizes memory usage by using an iterative deepening approach. This allows it to explore deeper into the search space without requiring significant memory resources.

1. **Memory Efficiency**:  
   IDA* reduces the need for extensive memory storage by performing a series of depth-limited searches, gradually increasing the depth limit until the solution is found. This makes it highly efficient for solving large puzzles, where A* might struggle due to its higher memory demands.

2. **Simulated Annealing for Exploration and Exploitation**:  
   Just like A*, IDA* adapts its search strategy based on a **temperature parameter**. The temperature controls the balance between **exploration** and **exploitation**:
   - **Higher temperature**: Increases the likelihood of exploration, allowing the algorithm to explore different parts of the search space and avoid getting stuck in local optima.
   - **Lower temperature**: Promotes exploitation, where the algorithm hones in on more promising paths and refines the search for the optimal solution.

In [ ]:

action_count_ida = 0
exploration_count_ida = 0
exploitation_count_ida = 0

TEMP_IDA = 0
COOLING_RATE_IDA = 0.99
MIN_T_IDA = 0.01
b = True


def solve_puzzle_ida(initial_state):
    """
    Risolve un puzzle sliding (es. 8-puzzle) usando l'algoritmo IDA*.
    """

    # size of the grid (e.g., 3 for a 3x3 puzzle)
    n = len(initial_state)

    # setupt the goal state
    goal_state = get_goal_state(n)

    # if the initial state is already the goal, return an empty path
    if initial_state == goal_state:
        return []

    # check if the puzzle is solvable; if not, return None
    if not is_solvable(initial_state, n):
        print("Il puzzle non è risolvibile!")
        return None
    
    # set the initial temperature for the exploration-exploitation heuristic
    global TEMP_IDA
    TEMP_IDA = 1000 * n

    print("Initial temperature: " + str(TEMP_IDA))
    print("Cooling rate: " + str(COOLING_RATE_IDA))
    print("Minimum temperature: " + str(MIN_T_IDA))

    # flatten the initial board for processing
    initial_state_flat = flatten(initial_state)

    # set the initial limit for the IDA* search
    limit = heuristic_ida(initial_state_flat, goal_state, n)

    visited = set()
    directions = { -1: 'left', 1: 'right', -n: 'up', n: 'down' }

    tracemalloc.start()
    print(f"Initial limit: {limit}")

    # Ciclo IDA*
    while True:

        # Start the IDA* search
        result, solution = ida_search(
            initial_state_flat, 0, limit, [], initial_state_flat.index(0),
            visited, goal_state, n, directions
        )

        # If the solution is found, return the path
        if solution is not None:
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            return solution, action_count_ida, exploration_count_ida, exploitation_count_ida, current

        # If no solution is found, return None (should never happen)
        if result == float('inf'):
            print("Nessuna soluzione trovata!")
            return None

        limit = result # Update the limit
        print(f"New limit: {limit}")


def ida_search(state, g, limit, path, empty_pos, visited, goal_state, n, directions):
    """
    IDA search
    state: current state
    g: cost so far
    limit: cost limit for this iteration
    path: path so far
    empty_pos: position of the empty tile
    visited: set of visited states
    goal_state: goal state
    n: size of the grid
    directions: dictionary of move directions
    """

    global action_count_ida
    action_count_ida += 1  # Increment the action count
    if action_count_ida % 500_000 == 0:
        current, peak = tracemalloc.get_traced_memory()
        print("N: " ,n," Action count: " + str(action_count_ida), " Current memory usage is " + str(current / 10**6) + "MB; Peak was " + str(peak / 10**6) + "MB")


    # cooling down the temperature
    global TEMP_IDA
    global b
    if TEMP_IDA > MIN_T_IDA:
        TEMP_IDA *= COOLING_RATE_IDA
    elif b:
        print("T < MIN_T, stop cooling down, action count: " + str(action_count_ida))
        b = False

    # Total estimated cost
    f = g + heuristic_ida(state, goal_state, n)

    # If the cost exceeds the limit, return the cost
    if f > limit:
        return f, None

    # If the current state is the goal state, return the cost and path
    if state == goal_state:
        return f, path

    # Set the state as visited
    visited.add(hash(tuple(state)))

    # Minimum cost for the next cycle
    min_cost = float('inf')

    # Set the position of the empty tile
    empty = empty_pos

    # Move the empty tile in all 4 directions: left, right, up, down
    for move in [-1, 1, -n, n]:
        new_empty = empty + move

        # if the move is invalid, skip it
        if not is_valid_move(empty, new_empty, n):
            continue

        # generate the new state by swapping the empty tile with the target tile
        new_state = state[:]
        new_state[empty], new_state[new_empty] = new_state[new_empty], new_state[empty]

        # if the new state has been visited, skip it
        if hash(tuple(new_state)) in visited:
            continue

        # generate the new path
        new_path = path + [directions[move]]

        # recursive call to IDA* with the new state
        result, solution = ida_search(
            new_state, g + 1, limit, new_path, new_empty, visited, goal_state, n, directions
        )

        if solution is not None:  # solution found
            return result, solution

        min_cost = min(min_cost, result)  # update the minimum cost

    visited.remove(hash(tuple(state)))  # Backtracking
    return min_cost, None  # return the minimum cost and None (no solution found)


def heuristic_ida(state, goal, n):
    """
    Heuristic function to estimate the cost from the current state to the goal state.
    use the exploration-exploitation heuristic if t > 0, else use only the exploitation heuristic.
    """

    global exploration_count_ida
    global exploitation_count_ida

    t = TEMP_IDA
    # use only the exploitation heuristic
    if t == 0: 
        return exploitation(state, goal, n)
    
    # use the exploration-exploitation heuristic
    r = random.random()
    # [0] = exploration, [1] = exploitation
    if t > r:
        exploration_count_ida += 1
        cost = exploration(state, goal, n)
    else:
        exploitation_count_ida += 1
        cost = exploitation(state, goal, n)
    
    return cost


def is_valid_move(empty, new_empty, n):
    """
    check if the move is valid
    """
    if new_empty < 0 or new_empty >= n * n:
        return False
    if empty % n == 0 and new_empty == empty - 1:
        return False
    if empty % n == n - 1 and new_empty == empty + 1: 
        return False
    return True



# Setup Functions Overview

The setup functions assist in the execution of the puzzle-solving process, handling essential tasks such as scrambling the puzzle, verifying correctness, and recording results. Here’s a summary of the key functions:

1. **random_swap**: Scrambles a given NxN puzzle board by applying a random number of swaps (between 800 and 1000 if no specific number is provided).
2. **write_file**: Saves the results and performance statistics (like execution time and moves) to a file for future analysis.
3. **is_correct**: Verifies if a board configuration matches the goal state after a series of moves.
4. **run_algorithm**: The main function that sets up the execution, tracks time, and writes the results to a file once the puzzle is solved.


In [ ]:
def random_swap(initial_board, N, N_MOVES = 0):
    """
    Perform random swaps on the initial board to generate a random board
    """
    moves = []

    # if no moves has been specified, generate a random number of moves
    if N_MOVES == 0:
        N_MOVES = random.randint(40 * (N**2), 50*(N**2))

    rand_board = initial_board.copy()

    # Find the position of the empty tile
    x, y = 0, 0
    for i in range(N):
        for j in range(N):
            if rand_board[i][j] == 0:
                x, y = i, j
                break
    
    direction = [((-1, 0), "up"), ((1, 0), "down"), ((0, -1), "left"), ((0, 1), "right")]
    prev_dx, prev_dy = 0, 0
    i = 0
    while i < N_MOVES:
        
        move = random.choice(direction)

        dx, dy = move[0]

        # Skip the move if it is the opposite of the previous move
        if (prev_dx + dx == 0) and (prev_dy + dy == 0):
            continue
        
        prev_dx, prev_dy = dx, dy
        new_x, new_y = x + dx, y + dy

        if new_x >= N or new_x < 0 or new_y >= N or new_y < 0:
            continue

        rand_board[x][y], rand_board[new_x][new_y] = rand_board[new_x][new_y], rand_board[x][y]
        x, y = new_x, new_y
        moves.append(move[1])

        i += 1

    return rand_board, moves

def get_correct_board(n):
    """ Return the correct board for the given size """
    nn = n * n
    numbers = list(range(1, nn))
    correct_board = [numbers[i:i+n] for i in range(0, nn-1, n)]
    correct_board[n-1].append(0)
    return correct_board

def print_board(board):
    """ Print the board in a readable format """
    for row in board:
        print("|", " | ".join(f"{x:3}" for x in row), "|")

def write_file(N, time_needed, number_of_rand_moves, quality, cost, percentage_exploration, percentage_exploitation, mem_usage, alg):
    """ Write the results to a CSV file """
    PATH_TO_FILE = "result.csv"
    SEPARATOR = ","
    with open(PATH_TO_FILE, "a") as file:
        str = f"{N}{SEPARATOR}"
        str += f"{time_needed}{SEPARATOR}"
        str += f"{number_of_rand_moves}{SEPARATOR}"
        str += f"{quality}{SEPARATOR}"
        str += f"{cost}{SEPARATOR}"
        str += f"{percentage_exploration}{SEPARATOR}"
        str += f"{percentage_exploitation}{SEPARATOR}"
        str += f"{mem_usage}{SEPARATOR}"
        str += f"{alg}\n"
        
        file.write(str)

def convert_seconds(seconds):
    """ Convert seconds to minutes and remaining seconds """
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return minutes, remaining_seconds

def make_move(board, direction, zero_pos):
    """ Move the empty tile in the given direction """
    # Extract current x, y coordinates of the zero
    x, y = zero_pos

    n = len(board)
    
    # Check direction and calculate new position of the zero
    direction = str(direction).lower()
    if direction == "left" and y > 0:  # Move left
        board[x][y], board[x][y - 1] = board[x][y - 1], board[x][y]
        zero_pos = (x, y - 1)  # Update zero_pos
    elif direction == "right" and y < n:  # Move right
        board[x][y], board[x][y + 1] = board[x][y + 1], board[x][y]
        zero_pos = (x, y + 1)  # Update zero_pos
    elif direction == "up" and x > 0:  # Move up
        board[x][y], board[x - 1][y] = board[x - 1][y], board[x][y]
        zero_pos = (x - 1, y)  # Update zero_pos
    elif direction == "down" and x < n:  # Move down
        board[x][y], board[x + 1][y] = board[x + 1][y], board[x][y]
        zero_pos = (x + 1, y)  # Update zero_pos
    else:
        print("Invalid move! ("+direction+")")
    return board, zero_pos

def apply_moves(board, moves):
    """ Apply a list of moves to the board """
    # Find the initial position of the zero
    zero_pos = None
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] == 0:
                zero_pos = (i, j)
                break
        if zero_pos:  # Exit early if found
            break

    # Apply all the moves in the list
    for move in moves:
        board, zero_pos = make_move(board, move, zero_pos)
    
    return board

def is_correct(start_board, moves):
    """ Check if the solution is correct """
    correct_board = get_correct_board(len(start_board))
    solution_board = apply_moves(start_board, moves)
    return correct_board == solution_board

def run_algorithm(N, initial_board = None, N_MOVES = 0):
    """ Run the algorithm for the given board """
    
    number_of_rand_moves = 0
    r = False
    correct_board = get_correct_board(N)

    # if the initial board is not provided, generate a random board
    if initial_board is None:
        print("Generating random board...")
        initial_board_rand, moves = random_swap(correct_board, N, N_MOVES)
        initial_board = initial_board_rand
        print("Initial board:")
        print_board(initial_board)

        number_of_rand_moves = len(moves)
        print("Number of random moves: ", number_of_rand_moves)

        r = True
    else:
        print("Initial board:")
        print_board(initial_board)

    # print some statistics
    goal_statate = get_goal_state(N)
    actual_state_list = flatten(initial_board)
    
    # Calculate the heuristic values
    manhattan_d = manhattan_distance(actual_state_list, goal_statate, N)
    tiles_out_of_place_d = tiles_out_of_place(actual_state_list, goal_statate, N)
    linear_conflict_d = linear_conflict(actual_state_list, goal_statate, N)

    print("Manhattan distance: ", manhattan_d)
    print("Tiles out of place: ", tiles_out_of_place_d)
    print("Linear conflict: ", linear_conflict_d)

    start_time = time.time()

    # if N >= 6, run the IDA* algorithm, else run the A* algorithm
    if N >= 6:
        print("\nRunning IDA* algorithm...")
        solution = solve_puzzle_ida(initial_board)
        alg = "IDA*"
    else:
        print("\nRunning A* algorithm...")
        solution = solve_puzzle_a(initial_board)
        alg = "A*"

    if solution is not None:

        # collect the results
        cost = solution[1]
        quality = len(solution[0])
        moves = solution[0]
        exploration_count = solution[2]
        exploitation_count = solution[3]
        mem_usage = round(solution[4] / 10**6, 2)
        efficiency = quality / cost

        if (exploration_count + exploitation_count) != 0:
            percentage_exploration = exploration_count / (exploration_count + exploitation_count)
            percentage_exploration = round(percentage_exploration, 3)
            percentage_exploitation = exploitation_count / (exploration_count + exploitation_count)
            percentage_exploitation = round(percentage_exploitation, 3)
        else:
            percentage_exploration = 0
            percentage_exploitation = 0

        print("Cost:", cost)
        print("Quality:", quality)
        print("Efficiency:", efficiency)
        print("Percantage of exploration:", percentage_exploration)
        print("Percantage of exploitation:", percentage_exploitation)

        if r is not True:
            print("Solution path:", moves)

        # Check if the solution is correct
        print("Verifying solution...")
        solution_board = apply_moves(initial_board, moves)
        if solution_board == correct_board:
            print("Solution board is correct!")
        else:
            print ("ERROR: Solution board is not correct!")
            
    # if no solution is found
    else:
        print("Puzzle is not solvable!")
        cost = -1
        quality = -1
        percentage_exploration = -1
        percentage_exploitation = -1


    # calculate the time needed
    end_time = time.time()
    time_needed = end_time - start_time
    time_needed = round(time_needed, 8) 
    minutes, seconds = convert_seconds(time_needed)
    print(f"Time needed: {minutes} minutes and {seconds} seconds")

    # write the results to a file
    if alg == "IDA*":
        write_file(N, time_needed, number_of_rand_moves, quality, cost, 0, 1, mem_usage, "IDA*")
    else:
        write_file(N, time_needed, number_of_rand_moves, quality, cost, percentage_exploration, percentage_exploitation, mem_usage, "A*")



# Main Function Overview

The behavior of the program is determined by the value of the variable **AUTO**. Depending on whether **AUTO** is set to `true` or `false`, the algorithm follows two different approaches:

1. **When AUTO is True**:  
   The program will automatically generate multiple puzzle boards of varying sizes (3x3, 4x4, 5x5, and 6x6) as specified in the code. For each board size, the algorithm performs a series of random swaps—between 800 and 1000 swaps—to scramble the board. Once scrambled, the algorithm attempts to solve each puzzle using the selected search strategy. This process is repeated for all board sizes, allowing the system to handle different puzzle complexities automatically.

2. **When AUTO is False**:  
   In this case, the algorithm is designed to solve a specific, pre-defined puzzle. The puzzle matrix is hardcoded into the program, and the algorithm will attempt to find a solution based on that fixed input, rather than generating new puzzles. This option is typically used for testing or solving a known instance of the puzzle.

### Note on Algorithm Selection and Performance

For puzzles sized **3x3 to 5x5**, the **A\* algorithm** will be used. These smaller puzzles do not require as much memory, and A* can efficiently solve them within reasonable time limits.

However, starting from **6x6 puzzles**, the memory requirements for A* increase significantly, especially depending on the number of swaps applied to the initial configuration. Due to this, **IDA\*** (Iterative Deepening A*) will be used for puzzles of size **6x6 and larger**. IDA* offers better memory management because it performs depth-limited searches, making it more suitable for handling larger, more complex puzzles.

Additionally, it's important to note that **more swaps** applied to the initial puzzle state will increase the solving time. Therefore, for **6x6 puzzles and larger**, it is recommended to limit the number of random moves to a maximum of **500** to ensure the algorithm runs efficiently and within a reasonable time frame (about 1 hour).

In [ ]:

# 6x6 puzzle
initial_board_6 = [
    [1,2,3,4,5,6],
    [7,8,9,10,11,12],
    [13,14,15,16,17,18],
    [19,20,21,22,23,24],
    [25,26,27,0,29,30],
    [31,32,33,28,34,35]
]

# 5x5 puzzle
initial_board_5 = [
    [1,2,3,4,5],
    [6,7,8,9,10],
    [11,12,13,14,15],
    [16,17,18,19,20],
    [21,22,23,0,24]
]

# 4x4 puzzle
initial_board_4 = [
    [5,15,8,6],
    [2,3,0,14],
    [10,12,7,11],
    [1,9,4,13]
]

initial_board = initial_board_4
N = len(initial_board)

# if you want to run the algorithm manually, set manual to True
# on manual mode, the initial board is used as the input
AUTO = False

N_MOVES_BIG_PUZZLE = 500

if not AUTO:
    times = []
    attempts = 1
    for i in range(attempts):
        print(i, "\n")
        start_time = time.time()
        run_algorithm(N, initial_board)
        execution_time = time.time() - start_time
        times.append(execution_time)
        
    print("Average execution time:", sum(times) / len(times))
else:
    attempts = 1
    N = [3,4,5,6]
    for i in N:
        title = "N: " + str(i)
        print("\n")
        print("+" + "-" * 55 + "+")
        print("|" + " " * 55 + "|")
        print("|" + f"{title :^55}" + "|")
        print("|" + " " * 55 + "|")
        print("+" + "-" * 55 + "+")

        for j in range(attempts):
            print("\n##### Iteration: ", j, " #####")
            if N == 6:
                run_algorithm(i, None, N_MOVES_BIG_PUZZLE)
            else:
                run_algorithm(i)
